# 用\_\_getattr\_\_、\_\_getattribute\_\_和\_\_setattr\_\_实现按需生成的属性

In [1]:
import logging

**示例：**把数据库的行表示为Python对象。

In [2]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value

In [3]:
data = LazyDB()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


访问data对象所缺失的foo属性，会导致Python调用刚才定义的\_\_getattr\_\_方法，从而修改实例的\_\_dict\_\_字典。

**改进：**给LazyDB添加记录功能，把程序对\_\_getattr\_\_的调用行为记录下来。

In [4]:
class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)

In [5]:
data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

exists: 5
Called __getattr__(foo)
foo:    Value for foo
foo:    Value for foo


## 需求变更1：在数据库中实现事务处理。用户下次访问某属性时，数据库中对应的行是否依然有效，已经相关事务是否依然处于开启状态。

**解决方法：**使用\_\_getattribute\_\_，在程序每次访问属性时，检查全局事务状态。

In [6]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

In [7]:
data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


In [8]:
try:
    class MissingPropertyDB(object):
        def __getattr__(self, name):
            if name == 'bad_name':
                raise AttributeError('%s is missing' % name)
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

    data = MissingPropertyDB()
    data.foo  # Test this works
    data.bad_name
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-56046b92ff89>", line 12, in <module>
    data.bad_name
  File "<ipython-input-8-56046b92ff89>", line 5, in __getattr__
    raise AttributeError('%s is missing' % name)
AttributeError: bad_name is missing


## hasatter和getattr函数的使用

hasatter函数：判断对象是否已经拥有了相关的属性。getattr函数：获取属性值。

**示例1：**\_\_getattr\_\_方法只调用了一次

In [9]:
data = LoggingLazyDB()
print('Before:     ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))
print('After:      ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))

Before:      {'exists': 5}
Called __getattr__(foo)
foo exists:  True
After:       {'exists': 5, 'foo': 'Value for foo'}
foo exists:  True


**示例2：**本类实现的是\_\_getattribute\_\_方法，每次在对象上面调用hasattr或getattr函数时，此方法都会执行。

In [10]:
data = ValidatingDB()
print('foo exists: ', hasattr(data, 'foo'))
print('foo exists: ', hasattr(data, 'foo'))

Called __getattribute__(foo)
foo exists:  True
Called __getattribute__(foo)
foo exists:  True


## 需求变更2：当程序把值赋给Python对象后，以惰性方式将其推回数据库。

In [11]:
class SavingDB(object):
    def __setattr__(self, name, value):
        # Save some data to the DB log
        super().__setattr__(name, value)

In [12]:
class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' % (name, value))
        super().__setattr__(name, value)

In [13]:
data = LoggingSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before:  {}
Called __setattr__(foo, 5)
After:   {'foo': 5}
Called __setattr__(foo, 7)
Finally: {'foo': 7}


**注意：**使用\_\_getattribute\_\_和\_\_setattr\_\_挂钩方法，每次访问对象属性事，它们都会触发。

**示例：**在查询对象的属性时，从对象内部的一份字典里面，搜寻与待查属性相关联的属性值。

In [14]:
class BrokenDictionaryDB(object):
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        return self._data[name]

In [15]:
# 会导致Python程序反复递归，从而堆栈溢出。
# try:
#     data = BrokenDictionaryDB({'foo': 3})
#     data.foo
# except:
#     logging.exception('Expected')
# else:
#     assert False

**解决办法：**采用super().\_\_getattribute\_\_方法，从实例的属性字典里面直接获取_data属性值，避免无限递归。

In [16]:
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

In [17]:
data = DictionaryDB({'foo': 3})
print(data.foo)

3
